<!--
Doc Writer email@nixdabei.de
v0.0.1, 2021-03-23
-->
[Home](../index.ipynb) / Mikrofon ADMP401: FFT Test
***
# Mikrofon ADMP401: FFT Test

In [22]:
%serialconnect

import math
from math import pi, cos, sin

class FFT:
    TWO_PI = 2 * pi
    
    def __init__( self, iSamples = 128 ):
        self.iSamples = iSamples
        
        self.yIn  = [0]*iSamples
        self.xOut = [0]*iSamples
        self.yOut = [0]*iSamples

        self.bufX = WindowedRingBuffer( iSamples )

    def fft( self, bufXIn, direction = 1 ):
        n = bufXIn.length
        factor = self.TWO_PI * direction

        for i in range( n ):
            self.xOut[i] = 0
            self.yOut[i] = 0

            arg = factor * i / n

            for k in range( n ):
                a = k * arg
                cosarg = cos(a);
                sinarg = sin(a);

                self.xOut[i] += ( bufXIn.get(k) * cosarg - self.yIn[k] * sinarg )
                self.yOut[i] += ( bufXIn.get(k) * sinarg + self.yIn[k] * cosarg )

        return self.yOut
    
# Usefull class to get elapsed time:
import time

class Timer():
    def __init__(self, name=None):
        self.name = name
#        self.getTime    = time.time_ns   # Python
#        self.TIME_SCALE = 10E9
        self.getTime    = time.ticks_ms # MicroPython
        self.TIME_SCALE = 1000
    def __enter__(self):
        self.start()
        
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.stop()

    def start( self ):
        self.t_start = self.getTime()

    def stop( self ):
        if self.name:
            print('[{}]'.format(self.name), end=" " )
            
        print('Total time: {} s'.format((self.getTime() - self.t_start)/self.TIME_SCALE))
        

class WindowedRingBuffer :
    def __init__(self,length):
        self.length   = length
        self.iLast    = length
        self.aData    = [0]*length # [i for i in range( length )]

        # Nuttall window, see https://en.wikipedia.org/wiki/Window_function#Hann_and_Hamming_windows
        self.aWindow  = [0.355768-0.487396*cos(2*pi*n/length)+0.144232*cos(4*pi*n/length)-0.012604*cos(6*pi*n/length) for n in range(length)] # Hanning window
                        # Hanning = 0.5 – 0.5*cos(2*pi * n / length )

    def get(self, iIndex):
        return self.aData[ (self.iLast + iIndex) % self.length ]*self.aWindow[ iIndex ]
        
    def add(self,val):
        self.iLast = (self.iLast - 1) % self.length
        self.aData[self.iLast] = val

    def __str__(self) :
        return( "{} {}".format( self.__class__, [ self.get(iIndex) for iIndex in range(self.length) ] ) )
    

from machine import ADC, Pin
import display
display = display.Display()

#adc = ADC(0)       # Croduino
adc = ADC( Pin(32) ) # HelTec, NodeMCU
adc.atten(ADC.ATTN_0DB)   # [1,1 V ~ 65535]

iCount = 0
FRAME_SIZE = 1

N_SAMPLE = 2**5
bufferX = WindowedRingBuffer( N_SAMPLE )
fft     = FFT( N_SAMPLE )

print( N_SAMPLE )
display.clear()

dX = int( 128/N_SAMPLE )

#SCALE = 63/2**12

NUMBER_LEDS = 16 # Half of N_SAMPLE !!!
PIN_LEDS = 14
DELTA = 4
import machine
import neopixel    
led = neopixel.NeoPixel(machine.Pin(PIN_LEDS), NUMBER_LEDS)        
while True:
    for i in range ( FRAME_SIZE ):
        iValue = adc.read_u16()  # read a raw analog value in the range 0-65535 (= 2^16 - 1)
        bufferX.add( adc.read_u16() )

    aFq = fft.fft( bufferX )
    
    # display.clear()
    
    for i in range( N_SAMPLE/2 ):
        #display.fill_rect( dX*i*2, int(63-aFq[i]*SCALE), dX*2, int(aFq[i]*SCALE), 1 )
        iR = min(255,max(0,(int(aFq[i]/32000*255 )) ) )
        # print( iR )
        led[ i ] = ( iR , 0, 0 )
        led.write()
    
    # display.show()
    # print( aFq )


Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.
32
.................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 116, in <module>
  File "<stdin>", line 32, in fft
KeyboardInterrupt: 


In [23]:
%serialconnect

from math import pi, cos, sin

class FFT:
    TWO_PI = 2 * pi
    
    def __init__( self, iSamples = 128 ):
        self.iSamples = iSamples
        
        self.yIn  = [0]*iSamples
        self.xOut = [0]*iSamples
        self.yOut = [0]*iSamples

        self.bufX = WindowedRingBuffer( iSamples )

    def fft( self, bufXIn, direction = 1 ):
        n = bufXIn.length
        factor = self.TWO_PI * direction

        for i in range( n ):
            self.xOut[i] = 0
            self.yOut[i] = 0

            arg = factor * i / n

            for k in range( n ):
                a = k * arg
                cosarg = cos(a);
                sinarg = sin(a);

                self.xOut[i] += ( bufXIn.get(k) * cosarg - self.yIn[k] * sinarg )
                self.yOut[i] += ( bufXIn.get(k) * sinarg + self.yIn[k] * cosarg )

        return self.yOut
    

class WindowedRingBuffer :
    def __init__(self,length):
        self.length   = length
        self.iLast    = length
        self.aData    = [0]*length # [i for i in range( length )]

        # Nuttall window, see https://en.wikipedia.org/wiki/Window_function#Hann_and_Hamming_windows
        self.aWindow  = [0.355768-0.487396*cos(2*pi*n/length)+0.144232*cos(4*pi*n/length)-0.012604*cos(6*pi*n/length) for n in range(length)] # Hanning window
                        # Hanning = 0.5 – 0.5*cos(2*pi * n / length )

    def get(self, iIndex):
        return self.aData[ (self.iLast + iIndex) % self.length ]*self.aWindow[ iIndex ]
        
    def add(self,val):
        self.iLast = (self.iLast - 1) % self.length
        self.aData[self.iLast] = val

    def __str__(self) :
        return( "{} {}".format( self.__class__, [ self.get(iIndex) for iIndex in range(self.length) ] ) )
    

from machine import ADC, Pin

adc = ADC( Pin(32) ) # HelTec, NodeMCU
adc.atten(ADC.ATTN_0DB)   # [1,1 V ~ 65535]

FRAME_SIZE = 1     # Number of readed values before next FFT
N_SAMPLE   = 2**5  # Width of FFT


bufferX = WindowedRingBuffer( N_SAMPLE )
fft     = FFT( N_SAMPLE )

NUMBER_LEDS = 16 # Half of N_SAMPLE !!!
PIN_LEDS = 14


import machine
import neopixel    

led = neopixel.NeoPixel(machine.Pin(PIN_LEDS), NUMBER_LEDS)        

SCALE = 255/32000

while True:
    for i in range ( FRAME_SIZE ):
        bufferX.add( adc.read_u16() ) # read a raw analog value in the range 0-65535 (= 2^16 - 1)

    aFq = fft.fft( bufferX )
    
    # display.clear()
    
    for i in range( NUMBER_LEDS ):
        iR = min(255,max(0,(int(aFq[i]*SCALE )) ) )
        led[ i ] = ( iR , 0, 0 )
        led.write()


Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.
................

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 84, in <module>
  File "<stdin>", line 31, in fft
KeyboardInterrupt: 
